In [1]:
#from solar_data_retrieval import retrieval
import pandas as pd 
import numpy as np
import sys
from sys import path
path.append('..')
from solardatatools import plot_2d
import s3fs
import boto3
from solardatatools.clear_day_detection import filter_for_sparsity
from solardataretrieval import utilities
from os.path import expanduser
home = expanduser('~')
with open(home + '/.aws/credentials') as f:
        lns = f.readlines()
        key = lns[3].split(' = ')[1].strip('\n')
        secret = lns[4].split(' = ')[1].strip('\n')
s3 = boto3.client('s3', aws_access_key_id=key,aws_secret_access_key=secret)


In [2]:
#set variables here: 
#choose how many sites you want in your data pipeline
number_of_sites = 10
number_of_days = 10
quantile_percent = 0.95

In [3]:
#download summary file from preprocessing
s3.download_file('pv.insight.sunpower.preprocessed','summary_file.csv','summary_file.csv')
sunpower_summary_file = pd.read_csv('summary_file.csv')

In [4]:
sunpower_summary_file_goodsparsity = sunpower_summary_file.loc[sunpower_summary_file['overall_spaersity'] < 0.3]
sunpower_summary_file_good = sunpower_summary_file_goodsparsity.loc[sunpower_summary_file_goodsparsity['overall_quality'] > 0.7]
sunpower_summary_file_good_288 = sunpower_summary_file_good.loc[sunpower_summary_file_good['time_sample'] == 288]


In [5]:
#choose site
all_sites = sunpower_summary_file_good_288.site_ID.unique()
site_IDs_selected = np.random.choice(all_sites, number_of_sites)

In [6]:
#choose sensor
index_to_download = [] #from original summary file! 
sensor_IDs_selected = []
for site_id in site_IDs_selected:
    site1 = sunpower_summary_file_good_288.loc[sunpower_summary_file_good_288['site_ID'] == site_id]
    sensor_IDs = site1.sensor_ID.unique()
    selected_sensor = np.random.choice(sensor_IDs, 1)[0]
    index_to_download.append(sunpower_summary_file.loc[(sunpower_summary_file['site_ID'] == site_id) & (sunpower_summary_file['sensor_ID'] == selected_sensor)].index.tolist()[0])
    sensor_IDs_selected.append(selected_sensor)

In [7]:
df_meta_data = pd.DataFrame(columns=['site_ID', 'sensor_ID', 'start_timestamp', 'end_timestamp', 'duration_days', 'time_sample', 'quantile_95', 'overall_spaersity', "overall_quality", "days_selected"])
counter = 1
for data_index in index_to_download:
    s3.download_file('pv.insight.sunpower.preprocessed','matrices/{0:04d}.npy'.format(data_index),'{0:04d}.npy'.format(data_index))
    power_signals_1 = np.load('{0:04d}.npy'.format(data_index))
    valid_indices = filter_for_sparsity(power_signals_1, solver='MOSEK')
    day_numbers = np.arange(power_signals_1.shape[1])
    good_day_numbers = day_numbers[valid_indices]
    list_of_selected_dates = np.random.choice(good_day_numbers, size=number_of_days, replace=False)
    power_signals = (power_signals_1 / np.quantile(power_signals_1, quantile_percent))
    power_signals_selected_days = power_signals[:,list_of_selected_dates]
    if data_index == index_to_download[0]:
        power_signals_selected_days_all = power_signals_selected_days
    else: 
        power_signals_selected_days = power_signals_selected_days
        power_signals_selected_days_all = np.concatenate([power_signals_selected_days_all, power_signals_selected_days], axis=1)
    df_meta_data = df_meta_data.append(sunpower_summary_file.iloc[data_index,:])
    df_meta_data.loc[data_index, 'days_selected'] = str(list_of_selected_dates)
    utilities.progress(counter, number_of_days, status='', bar_length=60)
    counter = counter + 1

In [8]:
df_data_input = pd.DataFrame(data=power_signals_selected_days_all[:])
upload_quantile_values = utilities.AWS_upload(df_meta_data,number_of_sites, number_of_days, "metadata")
upload_data = utilities.AWS_upload(df_data_input,number_of_sites, number_of_days, "data_input")
